In [917]:
import pandas as pd
import numpy as np
import math
import re
import seaborn as sns
import matplotlib as plt
from matplotlib import pyplot as plt

In [918]:
from sklearn.neighbors import KNeighborsRegressor

In [919]:
test =pd.read_csv('data/test.csv', index_col=0)
test_forest = pd.read_csv('data/test.csv', index_col=0)
train =pd.read_csv('data/train.csv', index_col=0)

In [920]:
test.shape

(1459, 79)

In [921]:
features_used = ['3SsnPorch', 'Bedroom/Bathroom', 'BedroomAbvGr', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'ExterQual',
       'Exterior1st', 'Exterior2nd', 'FireplaceQu', 'Fireplaces',
       'Functional', 'GarageArea', 'GarageCars', 'GarageFinish',
       'GarageYrBlt', 'GrLivArea', 'KitchenAbvGr', 'KitchenQual',
       'LotArea', 'LotConfig', 'LotFrontage', 'MasVnrArea', 'MasVnrType',
       'Neighborhood', 'OverallCond', 'OverallQual', 'SaleCondition',
       'ScreenPorch', 'TotRmsAbvGrd', 'TotalBath', 'TotalBsmtSF',
       'WoodDeckSF', 'YearRemodAdd']
1459-test.count()[features_used]

C:\Users\kobza\Anaconda3\lib\site-packages\pandas\core\series.py:1155: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


3SsnPorch             0.0
Bedroom/Bathroom      NaN
BedroomAbvGr          0.0
BsmtCond             45.0
BsmtExposure         44.0
BsmtFinType1         42.0
BsmtQual             44.0
ExterQual             0.0
Exterior1st           1.0
Exterior2nd           1.0
FireplaceQu         730.0
Fireplaces            0.0
Functional            2.0
GarageArea            1.0
GarageCars            1.0
GarageFinish         78.0
GarageYrBlt          78.0
GrLivArea             0.0
KitchenAbvGr          0.0
KitchenQual           1.0
LotArea               0.0
LotConfig             0.0
LotFrontage         227.0
MasVnrArea           15.0
MasVnrType           16.0
Neighborhood          0.0
OverallCond           0.0
OverallQual           0.0
SaleCondition         0.0
ScreenPorch           0.0
TotRmsAbvGrd          0.0
TotalBath             NaN
TotalBsmtSF           1.0
WoodDeckSF            0.0
YearRemodAdd          0.0
dtype: float64

In [922]:
#check for missingness
test[test.isna().any(axis=1)].loc[:, features_used]

C:\Users\kobza\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,3SsnPorch,Bedroom/Bathroom,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinType1,BsmtQual,ExterQual,Exterior1st,Exterior2nd,...,Neighborhood,OverallCond,OverallQual,SaleCondition,ScreenPorch,TotRmsAbvGrd,TotalBath,TotalBsmtSF,WoodDeckSF,YearRemodAdd
Id,,,,,,,,,,,,,,,,,,,,,
1461,0,NaN,2,TA,No,Rec,TA,TA,VinylSd,VinylSd,...,NAmes,6,5,Normal,120,5,NaN,882.0,140,1961
1462,0,NaN,3,TA,No,ALQ,TA,TA,Wd Sdng,Wd Sdng,...,NAmes,6,6,Normal,0,6,NaN,1329.0,393,1958
1463,0,NaN,3,TA,No,GLQ,Gd,TA,VinylSd,VinylSd,...,Gilbert,5,5,Normal,0,6,NaN,928.0,212,1998
1464,0,NaN,3,TA,No,GLQ,TA,TA,VinylSd,VinylSd,...,Gilbert,6,6,Normal,0,7,NaN,926.0,360,1998
1465,0,NaN,2,TA,No,ALQ,Gd,Gd,HdBoard,HdBoard,...,StoneBr,5,8,Normal,144,5,NaN,1280.0,0,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,0,NaN,3,TA,No,Unf,TA,TA,CemntBd,CmentBd,...,MeadowV,7,4,Normal,0,5,NaN,546.0,0,1970
2916,0,NaN,3,TA,No,Rec,TA,TA,CemntBd,CmentBd,...,MeadowV,5,4,Abnorml,0,6,NaN,546.0,0,1970
2917,0,NaN,4,TA,No,ALQ,TA,TA,VinylSd,VinylSd,...,Mitchel,7,5,Abnorml,0,7,NaN,1224.0,474,1996


In [923]:
# CAN'T DROP THESE IT IS PART OF OUR FEATURES

# test.drop(list(test[test['MasVnrArea'].isnull()].index), inplace=True)
# test_forest.drop(list(test_forest[test_forest['MasVnrArea'].isnull()].index), inplace=True)

In [924]:
test[test['LotConfig'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [925]:
test['LotConfig'].unique()

array(['Inside', 'Corner', 'FR2', 'CulDSac', 'FR3'], dtype=object)

In [926]:
train_frontage = train[~train['LotFrontage'].isnull()][['LotFrontage', 'LotArea', 'LotConfig']]
test_frontage = test[~test['LotFrontage'].isnull()][['LotFrontage', 'LotArea', 'LotConfig']]
test_missing = test[test['LotFrontage'].isnull()][['LotFrontage', 'LotArea', 'LotConfig']]
imputed_df = pd.DataFrame(columns=['LotFrontage', 'LotArea', 'LotConfig'])

for config in train['LotConfig'].unique():
    complete_subset_train = train_frontage[train_frontage['LotConfig'] == config]
    complete_subset_test = test_frontage[test_frontage['LotConfig'] == config]
    missing_subset = test_missing[test_missing['LotConfig'] == config]
    k = int(np.sqrt(complete_subset_train.shape[0] + missing_subset.shape[0]))
    clf = KNeighborsRegressor(k, weights='distance')
    trained_model = clf.fit(complete_subset_train['LotArea'].values.reshape(-1, 1), 
                            complete_subset_train['LotFrontage'])
    if missing_subset['LotArea'].values.reshape(-1, 1).shape[0] > 0:
        imputed_values = trained_model.predict(missing_subset['LotArea'].values.reshape(-1, 1))
        missing_subset['LotFrontage'] = imputed_values.reshape(-1)
    imputed_df = pd.concat([imputed_df,complete_subset_test,missing_subset])
imputed_df = imputed_df.sort_index()
test['LotFrontage'] = imputed_df['LotFrontage']
test['LotFrontage']

C:\Users\kobza\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\kobza\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\kobza\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

Id
1461     80.0
1462     81.0
1463     74.0
1464     78.0
1465     43.0
        ...  
2915     21.0
2916     21.0
2917    160.0
2918     62.0
2919     74.0
Name: LotFrontage, Length: 1459, dtype: float64

In [927]:
temp=list(test[test['BsmtExposure'].isnull() & test['BsmtQual'].notnull()].index)
#1488
#2349

In [928]:
import random
random.seed(0)
#obs1488 and 2349 is randomly assigned a a non-"No" categery for BsmtExposure
test['BsmtExposure'][1488]=test[test['BsmtExposure'].astype(str)!='No'].sample()['BsmtExposure']
test['BsmtExposure'][2349]=test[test['BsmtExposure'].astype(str)!='No'].sample()['BsmtExposure']

C:\Users\kobza\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\kobza\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [929]:
#ob with a BsmtFinSF2 but shows "no basement" in the BsmtFinType2 col NEED TO IMPUTE  - random
test[test['BsmtFinType2'].isnull() & test['BsmtFinSF2']!=0]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [930]:
# no need to impute BsmtFinType2, no missing obs in test set 
# random.seed(0)
# #obs333 is randomly assigned a non-"No" categery for BsmtFinType2
# train['BsmtFinType2'][333]=train[train['BsmtFinType2']!='nan'].sample()['BsmtFinType2'].values[0]

In [931]:
#ob with Na in Electrical 
test[test['Electrical'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [932]:
# no need to impute Electrical, no missing obs in test set 
# random.seed(0)
# #obs1380 is randomly assigned a random
# train['Electrical'][1380] = train['Electrical'].sample().values[0]

In [933]:
#for BsmtExposure - replace No with NoExposure; replace NaN with NoBsmt
test['BsmtExposure']=test['BsmtExposure'].str.replace('No','NoExposure')

In [934]:
test['BsmtExposure'].replace(np.NaN,'NoBsmt',inplace=True)

In [935]:
#GarageYrBlt - change No to numerical zero 
test['GarageYrBlt'].replace(np.NaN,0,inplace=True)

In [936]:
#cols for which NA will be replaced with "No" because NA is a true category thus need to change 
#to "No" so Pandas does not pick it up as np.NaN
#replacing fixing NA from only categorical Nominal
FixNA=['Alley', 'GarageType', 'MiscFeature']

In [937]:
#change np.NaN to "No"
for v in FixNA:
    test[v].replace(np.NaN,'No',inplace=True)

In [938]:
misc=test[['MiscFeature','MiscVal']]
misc=pd.pivot_table(misc,index='Id',columns='MiscFeature',values='MiscVal')
misc.replace(np.NaN,0,inplace=True)
misc=misc.drop(['No','Othr'],axis=1).apply(pd.to_numeric, errors='ignore', downcast='integer')

In [939]:
#combined misc with original DF and drop MiscFeature, MiscVal
test = pd.concat([test.drop(['MiscFeature','MiscVal'],axis=1),misc],axis=1)

In [940]:
#list of categorical variables 
cat_variables=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LotConfig', 'LandSlope','Neighborhood', 'Condition1', 'Condition2',
               'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 
               'Exterior2nd', 'MasVnrType', 'Foundation','BsmtExposure','BsmtFinType2',
               'Heating','CentralAir','Electrical', 'GarageType','SaleType', 'SaleCondition']


In [941]:
for v in cat_variables:
    print(test[v].value_counts())

RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64
Pave    1453
Grvl       6
Name: Street, dtype: int64
No      1352
Grvl      70
Pave      37
Name: Alley, dtype: int64
Reg    934
IR1    484
IR2     35
IR3      6
Name: LotShape, dtype: int64
Lvl    1311
HLS      70
Bnk      54
Low      24
Name: LandContour, dtype: int64
AllPub    1457
Name: Utilities, dtype: int64
Inside     1081
Corner      248
CulDSac      82
FR2          38
FR3          10
Name: LotConfig, dtype: int64
Gtl    1396
Mod      60
Sev       3
Name: LandSlope, dtype: int64
NAmes      218
OldTown    126
CollgCr    117
Somerst     96
Edwards     94
NridgHt     89
Gilbert     86
Sawyer      77
SawyerW     66
Mitchel     65
NWAmes      58
IDOTRR      56
Crawfor     52
BrkSide     50
Timber      34
NoRidge     30
StoneBr     26
SWISU       23
MeadowV     20
ClearCr     16
BrDale      14
NPkVill     14
Veenker     13
Blmngtn     11
Blueste      8
Name: Neighborhood, dtyp

In [942]:
feature_drop=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LandSlope','BldgType', 'HouseStyle','RoofStyle', 'RoofMatl',
              'BsmtFinType2','Heating','Electrical','Condition1', 'Condition2',
               'CentralAir','GarageType','SaleType','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
             '1stFlrSF','2ndFlrSF','LowQualFinSF']

#drop categorical features that present sparsity 
#MSZoning, Street, Alley, LotShape, LandCounter, Utilities, LandSlope, Condition 1,
#Condition 2, Roofstyle, RoofMatl, HouseStyle, #Heating #electrocial, SaleType
#centralair

#drop continuous variables that are redundant with total sqrt footage 
#BsmtFinSF2
#BsmtUnSF
#1stFlrSF
#2ndFlrSF
#LowQualFinSF
test=test.drop(feature_drop,axis=1)

In [943]:
train['PriceSqFt'] = train['SalePrice'] / train['GrLivArea']
hoodscore = train.groupby('Neighborhood').agg({'PriceSqFt': 'mean'}).sort_values(by='PriceSqFt', ascending=True).rank()
hoodscore['PriceSqFt']=hoodscore['PriceSqFt'].astype('int64')
#neighborhoods = avg_price_neighborhood.index.to_numpy()
#neighbscore=pd.DataFrame()
#train['Neighborhood'] = train['Neighborhood'].apply(lambda x: np.where(neighborhoods == x)[0][0] + 1)
#train = train.drop('PriceSqFt', axis=1)
#train['Neighborhood']
type(hoodscore)


pandas.core.frame.DataFrame

In [944]:
test=test.merge(hoodscore, left_on='Neighborhood', right_index=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'Neighborhood',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'PriceSqFt'],
      dtype='object')

In [945]:
test = test.sort_index()

In [946]:
test.drop('Neighborhood',axis=1,inplace=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'PriceSqFt'],
      dtype='object')

In [947]:
test.rename(columns={'PriceSqFt':'Neighborhood'}, inplace=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'Neighborhood'],
      dtype='object')

In [948]:
#Exterior1st - Assign other to categories with count under 100 
test['Exterior1st'].replace(['CemntBd','BrkFace','WdShing','Stucco','AsbShng','Stone',
                                     'BrkComm','AsphShn','CBlock','ImStucc'],'Other',inplace=True)


In [949]:
#Exterior2nd - Assign other to categories with count under 100 train['Exterior2nd'].replace(['CemntBd','BrkFace','WdShing','Stucco','AsbShng','Stone',

test['Exterior2nd'].replace(['CemntBd','Wd Shng','Stucco','BrkFace','AsbShng','Stone',
                                     'Brk Cmn','AsphShn','CBlock','ImStucc'],'Other',inplace=True)

In [950]:
test['Foundation'].replace(['Slab','Stone','Wood'],'Other',inplace=True)

In [951]:
LST = ('ExterQual','ExterCond','HeatingQC','KitchenQual')
LST1 = ('BsmtQual','BsmtCond','FireplaceQu','GarageQual','GarageCond')
LST2 = ('BsmtFinType1')
LST3 = ('Functional')
LST4 = ('GarageFinish')
LST5 = ('PavedDrive')
LST6 = ('PoolQC')
LST7 = ('Fence')


In [952]:
test[test['BsmtFullBath'].isna()]

,MSSubClass,LotFrontage,LotArea,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleCondition,Gar2,Shed,Neighborhood
Id,,,,,,,,,,,,,,,,,,,,,
2121,20,99.0,5940,FR3,4,7,1946,1950,MetalSd,Other,...,0,0,NaN,MnPrv,4,2008,Abnorml,0,0,8
2189,20,123.0,47007,Inside,5,7,1959,1996,Plywood,Plywood,...,0,0,NaN,NaN,7,2008,Normal,0,0,6


In [953]:
# replace two values with NA for basement bathrooms with 0
test['BsmtFullBath'].replace(np.NaN,0,inplace=True)
test['BsmtHalfBath'].replace(np.NaN,0,inplace=True)

In [954]:
test['TotalBath']=test['BsmtFullBath']+test['BsmtHalfBath']/2+test['FullBath']+test['HalfBath']/2
test['Bedroom/Bathroom']=test['BedroomAbvGr']/test['TotalBath']
test['Bedroom/Bathroom'].unique()


array([2.        , 1.2       , 1.        , 0.8       , 0.66666667,
       1.33333333, 1.5       , 0.85714286, 1.6       , 1.14285714,
       0.57142857, 0.4       , 3.        , 2.66666667, 0.5       ,
       1.66666667, 2.5       , 0.75      , 0.33333333, 0.88888889,
       0.44444444, 1.42857143, 4.        , 2.4       , 0.28571429,
       0.        , 1.11111111, 3.33333333])

In [955]:
for x in LST:
    test.loc[test[x] == 'Po',x] = 1
    test.loc[test[x] == 'Fa',x] = 2
    test.loc[test[x] == 'TA',x] = 3
    test.loc[test[x] == 'Gd',x] = 4
    test.loc[test[x] == 'Ex',x] = 5

In [956]:
for x in LST1:
    test[x].fillna(0, inplace=True)    
    test.loc[test[x] == 'Po',x] = 1
    test.loc[test[x] == 'Fa',x] = 2
    test.loc[test[x] == 'TA',x] = 3
    test.loc[test[x] == 'Gd',x] = 4
    test.loc[test[x] == 'Ex',x] = 5
    

C:\Users\kobza\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [957]:
# Nan values mean there are none for this there is no basement therefore value is 0
# all columns for ordinal has no basement or features if filled with 0
x = LST2
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Unf',x] = 1
test.loc[test[x] == 'LwQ',x] = 2
test.loc[test[x] == 'Rec',x] = 3
test.loc[test[x] == 'BLQ',x] = 4
test.loc[test[x] == 'ALQ',x] = 5
test.loc[test[x] == 'GLQ',x] = 6

In [958]:
x = LST3
test.loc[test[x] == 'Sal',x] = 1
test.loc[test[x] == 'Sev',x] = 2
test.loc[test[x] == 'Maj2',x] = 3
test.loc[test[x] == 'Maj1',x] = 4
test.loc[test[x] == 'Mod',x] = 5
test.loc[test[x] == 'Min2',x] = 6
test.loc[test[x] == 'Min1',x] = 6
test.loc[test[x] == 'Typ',x] = 6

In [959]:
x = LST4
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Unf',x] = 1
test.loc[test[x] == 'RFn',x] = 2
test.loc[test[x] == 'Fin',x] = 3

    

In [960]:
x = LST5
test.loc[test[x] == 'N',x] = 1
test.loc[test[x] == 'P',x] = 2
test.loc[test[x] == 'Y',x] = 3


In [961]:
x = LST6
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Fa',x] = 1
test.loc[test[x] == 'TA',x] = 2
test.loc[test[x] == 'Gd',x] = 3
test.loc[test[x] == 'Ex',x] = 4
    

In [962]:
x = LST7
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'MnWw',x] = 1
test.loc[test[x] == 'GdWo',x] = 2
test.loc[test[x] == 'MnPrv',x] = 3
test.loc[test[x] == 'GdPrv',x] = 4    

In [963]:
# These are the ordinal columns of the training data set
# train[['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtFinType1','HeatingQC','BsmtFullBath','KitchenQual',
#       'Functional',
# 'FireplaceQu','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','SalePrice']]
#dropping bathroom columns since we made a new column Total Bathroom
test = test.drop('BsmtFullBath',1)
test = test.drop('BsmtHalfBath',1)
test = test.drop('FullBath',1)
test = test.drop('HalfBath',1)

In [964]:
test[test.isna().any(axis=1)]

,MSSubClass,LotFrontage,LotArea,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,PoolQC,Fence,MoSold,YrSold,SaleCondition,Gar2,Shed,Neighborhood,TotalBath,Bedroom/Bathroom
Id,,,,,,,,,,,,,,,,,,,,,
1556,50,72.000000,10632,Inside,5,3,1917,1950,Wd Sdng,Wd Sdng,...,0,0,1,2010,Normal,0,0,16,1.5,2.000000
1692,60,94.057875,12891,Corner,8,5,2002,2002,VinylSd,VinylSd,...,0,0,7,2009,Normal,0,0,13,3.5,1.142857
1707,20,90.000000,7993,Inside,7,5,2008,2009,VinylSd,VinylSd,...,0,0,10,2009,Partial,0,0,22,2.0,1.500000
1883,60,70.000000,8749,Inside,7,5,2002,2002,VinylSd,VinylSd,...,0,0,11,2009,Normal,0,0,20,2.5,1.200000
1993,60,66.254764,7750,Inside,7,5,2002,2002,VinylSd,VinylSd,...,0,0,4,2008,Normal,0,0,13,2.5,1.200000
2005,20,87.000000,10037,Corner,8,5,2006,2007,VinylSd,VinylSd,...,0,0,8,2008,Normal,0,0,22,2.0,1.500000
2042,60,63.666667,7500,Inside,7,5,2002,2002,VinylSd,VinylSd,...,0,0,4,2008,Normal,0,0,22,2.5,1.200000
2121,20,99.000000,5940,FR3,4,7,1946,1950,MetalSd,Other,...,0,3,4,2008,Abnorml,0,0,8,1.0,2.000000
2152,30,85.000000,19550,Inside,5,7,1940,2007,NaN,NaN,...,0,0,1,2008,Normal,0,0,6,2.0,1.000000


In [965]:
test= test.drop('MSSubClass', 1)
test

,LotFrontage,LotArea,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrType,...,PoolQC,Fence,MoSold,YrSold,SaleCondition,Gar2,Shed,Neighborhood,TotalBath,Bedroom/Bathroom
Id,,,,,,,,,,,,,,,,,,,,,
1461,80.0,11622,Inside,5,6,1961,1961,VinylSd,VinylSd,None,...,0,3,6,2010,Normal,0,0,10,1.0,2.0
1462,81.0,14267,Corner,6,6,1958,1958,Wd Sdng,Wd Sdng,BrkFace,...,0,0,6,2010,Normal,12500,0,10,1.5,2.0
1463,74.0,13830,Inside,5,5,1997,1998,VinylSd,VinylSd,None,...,0,3,3,2010,Normal,0,0,13,2.5,1.2
1464,78.0,9978,Inside,6,6,1998,1998,VinylSd,VinylSd,BrkFace,...,0,0,6,2010,Normal,0,0,13,2.5,1.2
1465,43.0,5005,Inside,8,5,1992,1992,HdBoard,HdBoard,None,...,0,0,1,2010,Normal,0,0,25,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936,Inside,4,7,1970,1970,Other,CmentBd,None,...,0,0,6,2006,Normal,0,0,7,1.5,2.0
2916,21.0,1894,Inside,4,5,1970,1970,Other,CmentBd,None,...,0,0,4,2006,Abnorml,0,0,7,1.5,2.0
2917,160.0,20000,Inside,5,7,1960,1996,VinylSd,VinylSd,None,...,0,0,9,2006,Abnorml,0,0,17,2.0,2.0


In [966]:
test.to_csv('data/test_clean_forest.csv')

In [967]:
1459-test.count()[features_used]

3SsnPorch            0
Bedroom/Bathroom     0
BedroomAbvGr         0
BsmtCond             0
BsmtExposure         0
BsmtFinType1         0
BsmtQual             0
ExterQual            0
Exterior1st          1
Exterior2nd          1
FireplaceQu          0
Fireplaces           0
Functional           2
GarageArea           1
GarageCars           1
GarageFinish         0
GarageYrBlt          0
GrLivArea            0
KitchenAbvGr         0
KitchenQual          1
LotArea              0
LotConfig            0
LotFrontage          0
MasVnrArea          15
MasVnrType          16
Neighborhood         0
OverallCond          0
OverallQual          0
SaleCondition        0
ScreenPorch          0
TotRmsAbvGrd         0
TotalBath            0
TotalBsmtSF          1
WoodDeckSF           0
YearRemodAdd         0
dtype: int64

In [968]:
# Impute values for remaining missing values that are used in prediction
# Exterior1st: one missing value, impute randomly
# Exterior2nd: one missing value, impute randomly
# Functional: two missing values, impute randomly
# GarageArea: one missing value, impute randomly
# GarageCars: one missing vlaue, impute randomly
# KitchenQual: one missing value, impute randomly
# TotalBsmtSF: one missing value, impute randomly
# MasVnrType: 16 missing values, impute randomly
# MasVnrArea: 15 missing values, after random imputation above is done take the average by category of MasVnrType

random.seed(0)
df_na = test[['Exterior1st', 'Exterior2nd', 'Functional', 'GarageArea', 'GarageCars', 'KitchenQual', 
              'TotalBsmtSF', 'MasVnrType']]
df_na = df_na.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
test[df_na.columns] = df_na[df_na.columns]

In [969]:
test[test['MasVnrArea'].isna()]['MasVnrType']
test.loc[test['MasVnrType']=='None', 'MasVnrArea'] = 0
avg_by_type = test.groupby('MasVnrType').agg({'MasVnrArea': 'mean'})
test.loc[test['MasVnrArea'].isna(), :]
for van_type in test.loc[test['MasVnrArea'].isna(), 'MasVnrType'].unique():
    test.loc[(test['MasVnrType']==van_type) & test['MasVnrArea'].isna(), 'MasVnrArea'] = avg_by_type.loc[van_type, 'MasVnrArea']

In [970]:
1459-test.count()[features_used]

3SsnPorch           0
Bedroom/Bathroom    0
BedroomAbvGr        0
BsmtCond            0
BsmtExposure        0
BsmtFinType1        0
BsmtQual            0
ExterQual           0
Exterior1st         0
Exterior2nd         0
FireplaceQu         0
Fireplaces          0
Functional          0
GarageArea          0
GarageCars          0
GarageFinish        0
GarageYrBlt         0
GrLivArea           0
KitchenAbvGr        0
KitchenQual         0
LotArea             0
LotConfig           0
LotFrontage         0
MasVnrArea          0
MasVnrType          0
Neighborhood        0
OverallCond         0
OverallQual         0
SaleCondition       0
ScreenPorch         0
TotRmsAbvGrd        0
TotalBath           0
TotalBsmtSF         0
WoodDeckSF          0
YearRemodAdd        0
dtype: int64

In [971]:
cat_variables2=['LotConfig','Exterior1st', 'Exterior2nd', 'Foundation','MasVnrType',
               'SaleCondition','BsmtExposure']

In [972]:
#dummify categorical nominal and binary variables
for v in cat_variables2:
    temp = pd.get_dummies(test[v],prefix=v, prefix_sep='_')
    temp = temp.drop(temp.iloc[:,0:1],axis=1)
    test = pd.concat([test.drop(v,axis=1),temp],axis=1)

In [973]:
test.to_csv('data/test_clean.csv')

In [974]:
test

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,MasVnrType_Stone,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NoBsmt,BsmtExposure_NoExposure
Id,,,,,,,,,,,,,,,,,,,,,
1461,80.0,11622,5,6,1961,1961,0.0,3,3,3,...,0,0,0,0,1,0,0,0,0,1
1462,81.0,14267,6,6,1958,1958,108.0,3,3,3,...,0,0,0,0,1,0,0,0,0,1
1463,74.0,13830,5,5,1997,1998,0.0,3,3,4,...,0,0,0,0,1,0,0,0,0,1
1464,78.0,9978,6,6,1998,1998,20.0,3,3,3,...,0,0,0,0,1,0,0,0,0,1
1465,43.0,5005,8,5,1992,1992,0.0,4,3,4,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936,4,7,1970,1970,0.0,3,3,3,...,0,0,0,0,1,0,0,0,0,1
2916,21.0,1894,4,5,1970,1970,0.0,3,3,3,...,0,0,0,0,0,0,0,0,0,1
2917,160.0,20000,5,7,1960,1996,0.0,3,3,3,...,0,0,0,0,0,0,0,0,0,1
